In [1]:
import json
import re

# JSON 파일 로드
with open('merged_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 불필요한 필드 제거
def remove_unnecessary_fields(record):
    fields_to_remove = ['filename', 'original', 'questioner_ID', 'questioner_ISNI', 'answerer_ID', 'answerer_ISNI', 'context_learn']
    for field in fields_to_remove:
        record.pop(field, None)
    return record

data = [remove_unnecessary_fields(record) for record in data]

# 텍스트 정제 함수 정의
def clean_text(text):
    if not isinstance(text, str):
        return ''
    # 특수 문자 제거 (한글과 공백을 제외하지 않도록 수정)
    text = re.sub(r'[^\w\sㄱ-ㅎㅏ-ㅣ가-힣]', '', text)
    # 불필요한 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 텍스트 정제 적용
for record in data:
    record['context'] = clean_text(record.get('context', ''))
    # 질문과 답변의 comment 필드 처리
    question = record.get('question', {})
    answer = record.get('answer', {})
    question['comment'] = clean_text(question.get('comment', ''))
    answer['comment'] = clean_text(answer.get('comment', ''))
    record['question'] = question
    record['answer'] = answer

# 간단한 텍스트 요약 함수 정의
def summarize_text(text, max_sentences=3):
    sentences = re.split(r'(?<=[.!?])\s+', text)
    summary = ' '.join(sentences[:max_sentences])
    return summary

# 텍스트 요약 적용
for record in data:
    record['context_summary'] = summarize_text(record['context'])

# 전처리된 데이터를 JSON 파일로 저장
with open('processed_data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)
